# Analysing the cleaned OCOD dataset

This is causing problems and needs to be fixed needs the preceeding code
'hillfield mansions and tagore house'	

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
import numpy as np
import os
import re
import io
import zipfile
from helper_functions import *



In [2]:
#!pip install thefuzz[speedup]
#from thefuzz import process #an alternative could jellyfish, nltk, diff-match-patch

In [3]:
target_post_area = "Data/ONSPD_NOV_2021_UK.csv"

with zipfile.ZipFile("/tf/empty_homes_data/" + "ONSPD_NOV_2021_UK.zip") as zf:
    with io.TextIOWrapper(zf.open(target_post_area), encoding = 'latin-1') as f:
        postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','oa11','lsoa11', 'msoa11', 'ctry']]
        postcode_district_lookup = postcode_district_lookup[(postcode_district_lookup['ctry'] == 'E92000001') | (postcode_district_lookup['ctry'] == 'W92000004')]
        postcode_district_lookup.rename(columns = {'pcds':'postcode2',
                                                  'oslaua':'lad11cd',
                                                   'oa11':'oa11cd',
                                                  'lsoa11': 'lsoa11cd',
                                                  'msoa11':'msoa11cd'}, inplace = True)
        #spaces are removed because I don't know if the formatting is the same in the two datasets
        postcode_district_lookup['postcode2']= postcode_district_lookup['postcode2'].str.lower().str.replace(r"\s", r"", regex = True)
        postcode_district_lookup.drop('ctry',  axis =1, inplace = True)


<ipython-input-3-e74e004024cc>:5: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','oa11','lsoa11', 'msoa11', 'ctry']]


In [4]:
ocod_data =  pd.read_csv("/tf/empty_homes_data/OCOD_cleaned_expanded2.csv")

##add in the geographic area data like lsoa etc
ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")

ocod_data = ocod_data.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

ocod_data['street_name'] = ocod_data['street_name'].str.replace(r"^ +| +$", r"", regex=True)
ocod_data['street_number2'] = ocod_data['street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)

#This stripped out versionof street name is used several times throughout the notebook
ocod_data['street_name2'] = ocod_data.loc[:,'street_name'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

<ipython-input-4-4bb3ddb47f6f>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")
<ipython-input-4-4bb3ddb47f6f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ocod_data['street_number2'] = ocod_data['street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


## Using price paid data to match names

The land registry does not use standardised LAD codes or names and 
the LAD names it uses appear to be wrong sometimes. I need to know the LADs so that I only try road matching within local authorities to minimise the chance of having the same road twice. To get around this I will use the substantially larger database of the price paid data to get all the land registry district names and match them to the onsp using the postcodes. This works as there are a large number of sales in each district most of them will have a postcode. There are cases where the wrong district or postcode is applied meaning a single district name can have two or more lad11cd's, to solve this I simply take the lad11cd with the largest number of counts.

The resulting OCOD data frame has a LAD11CD for each entry, and thus allows the fuzzy road matching to work effectively

In [5]:
#https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd
price_paid_headers = ['Transaction unique identifier', 'Price', 'Date of Transfer', 'Postcode', 'Property Type', 
                     'Old New', 'Duration', 'PAON', 'SAON', 'Street',  'Locality', 'Town', 'District', 'County',
                     'PPD Category Type', 'Record Status - monthly file only']
#clean up to make working with them easier
price_paid_headers = [x.lower().replace(' ', '_') for x in price_paid_headers]

##
## Here we add in the lsoa11cd msoa11cd and lad11cd using postcode
##

#price_paid_df = pd.read_csv('/tf/empty_homes_data/price_paid_files/pp-2021.csv', names = price_paid_headers)
price_paid_df = pd.concat([ pd.read_csv('/tf/empty_homes_data/price_paid_files/'+x, names = price_paid_headers) for x in os.listdir('/tf/empty_homes_data/price_paid_files')])

price_paid_df['street'] = price_paid_df['street'].str.lower()

price_paid_df['street_name2'] = price_paid_df.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

price_paid_df['locality'] = price_paid_df['locality'].str.lower()
price_paid_df['paon'] = price_paid_df['paon'].str.lower()

price_paid_df = clean_street_numbers(price_paid_df, original_column = 'paon')

price_paid_df['postcode2'] = price_paid_df['postcode'].str.lower().str.replace(r"\s", r"", regex=True)

price_paid_df = price_paid_df.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

##
## Not all observations have a postcode and some postcodes are erroneous or otherwise cannot be found in the postcode database
## These entries need to be found ny joining districts
##

#when there are multiples take the lad11cd with the largest number of counts
lad_lookup = price_paid_df[['district', 'lad11cd']].dropna().groupby(['district', 'lad11cd']).size().reset_index()
lad_lookup.rename(columns = {0:'counts'}, inplace = True)
lad_lookup = lad_lookup.sort_values('counts', ascending=False).groupby('lad11cd').first().reset_index()
lad_lookup.drop('counts', axis = 1, inplace = True)

temp = ocod_data

#only take data that is missing the lad code. If you include only is missing postcode, bad postcode entries also get excluded leaving a small number of
## blanks in the final dataset
temp = temp[temp['lad11cd'].isna()]

temp = temp.drop('lad11cd', axis = 1)

temp = temp.merge(lad_lookup, left_on = "district", right_on = "district")

temp['lad11cd'].isna().sum() #there are no na values showing all districts now have a lad code

#join the ocod data back together again
ocod_data = pd.concat( [temp, ocod_data[~ocod_data['lad11cd'].isna()]])

#tidy up
#del temp
#del lad_lookup
#del price_paid_df

In [6]:
price_paid_df

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,old_new,duration,paon,saon,street,...,county,ppd_category_type,record_status_-_monthly_file_only,street_name2,street_number,postcode2,lad11cd,oa11cd,lsoa11cd,msoa11cd
0,{64342BFF-702A-422C-E053-6C04A8C0FB8A},555000,2017-09-18 00:00,PO19 1LB,O,N,L,18,NaN,north street,...,WEST SUSSEX,B,A,northstreet,18,po191lb,E07000225,E00160587,E01031490,E02006570
1,{64342BFF-702B-422C-E053-6C04A8C0FB8A},25000,2017-09-29 00:00,NaN,O,Y,L,ifield car park,PLOT 55,charlwood road,...,WEST SUSSEX,B,A,charlwoodroad,NaN,NaN,NaN,NaN,NaN,NaN
2,{64342BFF-702C-422C-E053-6C04A8C0FB8A},177500,2017-04-07 00:00,PO19 7BJ,O,N,F,glenmore business park portfield works,UNIT 6,chichester by pass,...,WEST SUSSEX,B,A,chichesterbypass,NaN,po197bj,E07000225,E00160721,E01031513,E02006572
3,{64342BFF-702D-422C-E053-6C04A8C0FB8A},90000,2017-10-09 00:00,BN43 5DB,O,N,L,118,NaN,high street,...,WEST SUSSEX,B,A,highstreet,118,bn435db,E07000223,E00167749,E01031364,E02006538
4,{64342BFF-702E-422C-E053-6C04A8C0FB8A},190000,2017-10-10 00:00,PO21 1QD,O,N,F,25,NaN,station road,...,WEST SUSSEX,B,A,stationroad,25,po211qd,E07000224,E00160299,E01031436,E02006558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4812242,{C3C3F9B5-7995-362B-E053-6B04A8C03ACC},220000,2021-03-08 00:00,TR11 2RR,T,N,F,40,NaN,pengarth rise,...,CORNWALL,A,A,pengarthrise,40,tr112rr,E06000052,E00094935,E01018806,E02003916
4812243,{C3C3F9B5-7996-362B-E053-6B04A8C03ACC},430000,2021-02-05 00:00,PL15 8SS,D,N,F,17,NaN,well meadow,...,CORNWALL,A,A,wellmeadow,17,pl158ss,E06000052,E00095795,E01018968,E02003934
4812244,{C3C3F9B5-7997-362B-E053-6B04A8C03ACC},236500,2021-03-12 00:00,TR21 0NE,F,N,L,mount flagon,ANNETT,NaN,...,ISLES OF SCILLY,A,A,NaN,NaN,tr210ne,E06000053,E00096403,E01019077,E02006781
4812245,{C3C3F9B5-7998-362B-E053-6B04A8C03ACC},185050,2021-03-09 00:00,TR27 4DJ,T,N,F,102,NaN,commercial road,...,CORNWALL,A,A,commercialroad,102,tr274dj,E06000052,E00095885,E01018982,E02003944


## Load Voa and get business postcodes

This alllows businesses to be identified, and also missing lsoa to be added via the street name

In [7]:
VOA_headers_raw= ["Incrementing Entry Number", "Billing Authority Code", "NDR Community Code", 
 "BA Reference Number", "Primary And Secondary Description Code", "Primary Description Text",
"Unique Address Reference Number UARN", "Full Property Identifier", "Firms Name", "Number Or Name",
"Street", "Town", "Postal District", "County", "Postcode", "Effective Date", "Composite Indicator",
 "Rateable Value", "Appeal Settlement Code", "Assessment Reference", "List Alteration Date", "SCAT Code And Suffix",
 "Sub Street level 3", "Sub Street level 2", "Sub Street level 1", "Case Number", 
 "Current From Date", "Current To Date", 
]

#set to lower and replace spaces with underscore to turn the names into appropriate column names
VOA_headers = [x.lower().replace(" ", "_") for x in VOA_headers_raw]


voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +
                    'uk-englandwales-ndr-2017-listentries-compiled-epoch-0029-baseline-csv.csv',
                   sep = "*",
                   encoding_errors= 'ignore',
                    header=None,
                   names = VOA_headers,
                    index_col = False,
                    #usecols = list(range(1,28))
                   )
voa_businesses['postcode'] = voa_businesses['postcode'].str.lower()
voa_businesses['street'] = voa_businesses['street'].str.lower()

#voa_businesses['street_name2'] = voa_businesses['street'].str.replace(r"'", "", regex = True).\
#    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
voa_businesses['street_name2'] = voa_businesses.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)

#this removes advertising hordings which are irrelevant
voa_businesses = voa_businesses.loc[voa_businesses['primary_description_text'].str.contains("ADVERTISING")==False,:]
#remove several kinds of car parking space
voa_businesses = voa_businesses[~voa_businesses['primary_and_secondary_description_code'].isin(['C0', 'CP','CP1', 'CX', 'MX'])]
##
##
## Warning this removes a large amount of columns, these may be interesting for some people
##
##
voa_businesses = voa_businesses.iloc[:,4:15]
#Extract the street number
#replace unit numbers with nothing to avoid accidentally using them as street numbers
#voa_businesses['street_number'] = voa_businesses['number_or_name'].str.replace(r"(unit|suite)\s\d+(,)?", "", regex = True).str.extract(r"(\b[0-9\-]+$)")
voa_businesses = clean_street_numbers(voa_businesses, original_column = 'number_or_name')

#Westfield has a ludicrous numbering system and is removed to avoid issues
voa_businesses.loc[voa_businesses['full_property_identifier'].str.contains('WESTFIELD SHOPPING CENTRE')==True, 'street_number']= np.nan
#sometimes a phone number ends up in the address causing havoc, these are few (145) and  are ignored
voa_businesses.loc[voa_businesses['street_number'].str.contains(r'-\d+-', regex = True)==True, 'street_number']= np.nan

voa_businesses['postcode2'] = voa_businesses['postcode'].str.lower().str.replace("\s", "", regex = True)
#Extracts the name of any "house" buildings
voa_businesses['building_name'] = voa_businesses['number_or_name'].str.lower().str.extract(r'((?<!\S)(?:(?!\b(?:\)|\(|r\/o|floor|floors|pt|and|annexe|room|gf|south|north|east|west|at|on|in|of|adjoining|adj|basement|bsmt|fl|flr|flrs|wing)\b)[^\n\d])*? house\b)')

#add in postcode data and LSOA etc data, this is useful for a range of tasks
voa_businesses = voa_businesses.merge(postcode_district_lookup, left_on = 'postcode2', right_on = "postcode2")



voa_businesses['street_name2'] = voa_businesses.loc[:,'street'].str.replace(r"'", "", regex = True).\
    str.replace(r"s(s)?(?=\s)", "", regex = True).str.replace(r"\s", "", regex = True)
#del voa_businesses

#this is quite large and no longer necessary so is removed from the workspace to save memory
#del postcode_district_lookup

<ipython-input-7-aaf34f73e94c>:14: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv('/tf/empty_homes_data/' +


# Street and buildings to match lsoa

This section fills in missing lsoa11cd using knowledge of the LAD11cd and the streets within it. This takes data from price paid and voa

In [8]:
##
##This process is quite convoluted and there is certainly a more efficient and pythonic way
## however the order within each filling method is important to ensure that there are no duplicates
## as this causes the OCOD dataset to grow with duplicates
##

#replce the missing lsoa using street matching

temp_lsoa = pd.concat([
    price_paid_df[['street_name2', 'lad11cd', 'lsoa11cd']], voa_businesses[['street_name2', 'lad11cd', 'lsoa11cd']]
                      ]).dropna(axis = 0, how = 'any', inplace = False)

temp = temp_lsoa.groupby(['street_name2', 'lad11cd', 'lsoa11cd']).size().reset_index().groupby(['street_name2', 'lad11cd']).size()\
.reset_index().rename(columns = {0:'counts'})

temp = temp[temp['counts']==1].merge(temp_lsoa.drop_duplicates(), 
                                     how = "left", on = ['street_name2', 'lad11cd']).rename(columns ={'lsoa11cd':'lsoa_street'})

ocod_data = ocod_data.merge(temp[['lsoa_street', "street_name2", "lad11cd"]], how = "left", on = ['street_name2', 'lad11cd'])


#replace the missing lsoa using building matching

temp = price_paid_df.copy()

temp['paon'] = temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)

temp = temp.groupby(['paon', 'lad11cd', 'lsoa11cd']).size().reset_index()

temp = temp[temp['paon'].str.len()!=0].groupby(['paon', 'lad11cd']).size().reset_index().rename(columns = {0:'counts'})

pp_temp = price_paid_df[['paon', 'lad11cd','oa11cd' , 'lsoa11cd']]

pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)

pp_temp = pp_temp.drop_duplicates()

temp = temp[temp['counts']==1].merge(pp_temp, 
                                     how = "left", on = ['paon', 'lad11cd'])

temp = temp.rename(columns ={'lsoa11cd':'lsoa_building',
                            'oa11cd':'oa_building',
                            'paon':'building_name'}).drop_duplicates(subset = ['building_name', 'lsoa_building'])

ocod_data = ocod_data.merge(temp[['lsoa_building', "oa_building" ,"building_name", "lad11cd"]], 
                       how = "left", 
                       on =  ['building_name', 'lad11cd'])

ocod_data = ocod_data.merge(voa_businesses.loc[~voa_businesses['building_name'].isna(), 
                   ['building_name','oa11cd', 'lsoa11cd', 'lad11cd']].drop_duplicates(subset = ['building_name', 'lsoa11cd']).rename(
    columns = {'lsoa11cd':'lsoa_busi_building',
              'oa11cd':'oa_busi_building'}),
                       how = "left", 
                       on =  ['building_name', 'lad11cd'])


<ipython-input-8-4b6113370a7f>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  temp['paon'] = temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
<ipython-input-8-4b6113370a7f>:34: FutureWarning: The default value of regex will change from True to False in a future version.
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)
<ipython-input-8-4b6113370a7f>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_temp['paon'] = pp_temp['paon'].str.replace(r"\d+", "").replace(r"and|\&|-|,", "", regex = True).replace(r"^ +| +$", r"", regex=True)


In [9]:
#I appreciate this is an almost artistically silly way of doing this. 
for x in ['lsoa_street', 'lsoa_building', 'lsoa_busi_building']:#, 'lsoa_business']:
    ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data[x][ocod_data['lsoa11cd'].isnull()] 
    
#add in the OA for when building matches have occured
for x in ['oa_building', 'oa_busi_building']:#, 'lsoa_business']:
    ocod_data.loc[ocod_data['oa11cd'].isnull(), 'oa11cd'] = ocod_data[x][ocod_data['oa11cd'].isnull()] 


In [10]:
ocod_data['lsoa11cd'].isnull().sum()/ocod_data.shape[0]

0.17601503759398496

In [11]:
#after all other lsoa adding methods are completed
#all nested properties with missing lsoa have the lsoa of the other properties within their group added

temp = ocod_data.loc[(ocod_data['lsoa11cd'].notnull()) & (ocod_data['within_larger_title']==True) ,['lsoa11cd', 'title_number']].\
groupby(['lsoa11cd', 'title_number']).size().reset_index()
temp = temp[['lsoa11cd', 'title_number']].rename(columns = {'lsoa11cd':'lsoa_nested'})

#there are a small number of nested addresses where there are multiple lsoa this prevents increasing the number of observations with these duplicates
#I don't think it matters if a ver observations are in neighbouring lsoa, the general spatial coherence is maintained
temp = temp.groupby('title_number')['lsoa_nested'].first().reset_index()


ocod_data = ocod_data.merge(temp[['title_number', 'lsoa_nested']], 
                       how = "left",
                           on = "title_number")

ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data['lsoa_nested'][ocod_data['lsoa11cd'].isnull()] 

##
##repeats again but for oa instead of lsoa
##

temp = ocod_data.loc[(ocod_data['oa11cd'].notnull()) & (ocod_data['within_larger_title']==True) ,['oa11cd', 'title_number']].\
groupby(['oa11cd', 'title_number']).size().reset_index()
temp = temp[['oa11cd', 'title_number']].rename(columns = {'oa11cd':'oa_nested'})

#there are a small number of nested addresses where there are multiple lsoa this prevents increasing the number of observations with these duplicates
#I don't think it matters if a ver observations are in neighbouring lsoa, the general spatial coherence is maintained
temp = temp.groupby('title_number')['oa_nested'].first().reset_index()


ocod_data = ocod_data.merge(temp[['title_number', 'oa_nested']], 
                       how = "left",
                           on = "title_number")

ocod_data.loc[ocod_data['oa11cd'].isnull(), 'oa11cd'] = ocod_data['oa_nested'][ocod_data['oa11cd'].isnull()] 


## Matching at sub street level

Some streets are on the boundary of LSOA this section uses the street number to match to the nearest lsoa.

In [12]:
filled_lsoa_list = []

unique_lad_codes = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull()]['lad11cd'].unique()

for target_lad in unique_lad_codes:
    print(target_lad)
    #subset to the relevat rows within a single lad
    missing_lsoa_df = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull() & (ocod_data['lad11cd']==target_lad)]
    missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)

    target_street_names = missing_lsoa_df['street_name2'].unique()

    temp_lsoa = pd.concat([
        #the price paid data with names changed
        price_paid_df[price_paid_df['street_name2'].isin(target_street_names )  & 
                                    (price_paid_df['lad11cd']==target_lad) ], 
      #voa data added in                
                      voa_businesses[(voa_businesses['lad11cd']==target_lad)]]
                                                    )[['street_name2', 'street_number', 'lsoa11cd', 'lad11cd']].dropna(axis = 0, how = 'any', inplace = False)

    temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "", regex = True).str.replace(r"[^\d]", "", regex = True)
    
    temp_lsoa  = create_all_street_addresses(temp_lsoa[temp_lsoa['street_name2'].isin(target_street_names ) & 
                                            temp_lsoa['street_number2'].notnull()], 
                                       target_lad, 
                                       ['street_name2', 'street_number2', 'lsoa11cd'])
    

    for target_road in target_street_names:
        print(target_road)
        missing_lsoa_road = missing_lsoa_df[missing_lsoa_df['street_name2']== target_road ]
        temp_road = temp_lsoa[temp_lsoa['street_name2'] ==target_road ]

        if len(temp_road)>0:
            missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2'])) 
                                             for missing_lsoa_row 
                                             in range(0, len(missing_lsoa_road))]
            filled_lsoa_list = filled_lsoa_list + [missing_lsoa_road]
 
#join the list back together
temp_lsoa = pd.concat(filled_lsoa_list)

#join the ocod dataset backtogether
ocod_data = pd.concat([ocod_data[~ocod_data['unique_id'].isin(temp_lsoa['unique_id'])], temp_lsoa ] )
ocod_data


E06000031


<ipython-input-12-8bc6428979cb>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-12-8bc6428979cb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
broadway
E07000032
albionclose
colliergrove
gadfieldgrove
E09000001


<ipython-input-12-8bc6428979cb>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-12-8bc6428979cb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.002
londonwall
cheapside
woodstreet
colemanstreet
princestreet


...

<ipython-input-12-8bc6428979cb>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-12-8bc6428979cb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)


byerlane
thesquare
E07000166


<ipython-input-12-8bc6428979cb>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2']))
<ipython-input-12-8bc6428979cb>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "").str.replace(r"[^\d]", "", regex = True)
<ipython-input-12-8bc6428979cb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
marketplace


,Unnamed: 0,title_number,within_title_id,unique_id,within_larger_title,tenure,unit_id,unit_type,building_name,street_number,...,street_number2,street_name2,lad11cd,lsoa_street,lsoa_building,oa_building,oa_busi_building,lsoa_busi_building,lsoa_nested,oa_nested
0,0,CB400630,1,CB400630-1,True,Freehold,NaN,NaN,NaN,2,...,2,millerway,E06000031,E01015611,NaN,NaN,NaN,NaN,E01015611,NaN
1,1,CB400630,2,CB400630-2,True,Freehold,NaN,NaN,NaN,4,...,4,millerway,E06000031,E01015611,NaN,NaN,NaN,NaN,E01015611,NaN
2,2,CB400630,3,CB400630-3,True,Freehold,NaN,NaN,NaN,6,...,6,millerway,E06000031,E01015611,NaN,NaN,NaN,NaN,E01015611,NaN
3,3,CB400630,4,CB400630-4,True,Freehold,NaN,NaN,NaN,8,...,8,millerway,E06000031,E01015611,NaN,NaN,NaN,NaN,E01015611,NaN
4,4,CB400630,5,CB400630-5,True,Freehold,NaN,NaN,NaN,10,...,10,millerway,E06000031,E01015611,NaN,NaN,NaN,NaN,E01015611,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57055,27785,NT214819,1,NT214819-1,False,Freehold,NaN,NaN,NaN,27 to 33,...,33,highroad,E07000172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57106,66326,ON137736,1,ON137736-1,False,Freehold,NaN,NaN,NaN,2-8,...,8,highstreet,E07000181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57126,10554,NT73623,1,NT73623-1,False,Freehold,NaN,NaN,NaN,22,...,22,churchstreet,E07000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57306,20461,SY64666,1,SY64666-1,False,Freehold,NaN,NaN,NaN,8 to 14,...,14,thesquare,E07000215,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Doing this grouped nested business a second time pushes lsoa ID over 90% which seems good enough for me

#after all other lsoa adding methods are completed
#all nested properties with missing lsoa have the lsoa of the other properties within their group added

temp = ocod_data.loc[(ocod_data['lsoa11cd'].notnull()) & (ocod_data['within_larger_title']==True) ,['lsoa11cd', 'title_number']].\
groupby(['lsoa11cd', 'title_number']).size().reset_index()
temp = temp[['lsoa11cd', 'title_number']].rename(columns = {'lsoa11cd':'lsoa_nested2'})

#there are a small number of nested addresses where there are multiple lsoa this prevents increasing the number of observations with these duplicates
#I don't think it matters if a ver observations are in neighbouring lsoa, the general spatial coherence is maintained
temp = temp.groupby('title_number')['lsoa_nested2'].first().reset_index()


ocod_data = ocod_data.merge(temp[['title_number', 'lsoa_nested2']], 
                       how = "left",
                           on = "title_number")

ocod_data.loc[ocod_data['lsoa11cd'].isnull(), 'lsoa11cd'] = ocod_data['lsoa_nested2'][ocod_data['lsoa11cd'].isnull()] 

In [14]:
#percent of dataset without lsoa
ocod_data['lsoa11cd'].isnull().sum()/ocod_data.shape[0]

0.09225563909774435

## Add in counts of businesses per oa and LSOA

In [15]:
#Create a dataframe that contains the counts of businesses per OA
postcode_counts_voa = voa_businesses.groupby('oa11cd').size().reset_index(name = 'business_counts')
ocod_data = pd.merge(ocod_data, postcode_counts_voa, on = "oa11cd", how = "left")
ocod_data["business_counts"] = ocod_data["business_counts"].fillna(0)

#do the same for lsoa
lsoa_counts_voa = voa_businesses.groupby('lsoa11cd').size().reset_index(name = 'lsoa_business_counts')
ocod_data = pd.merge(ocod_data, lsoa_counts_voa, on = "lsoa11cd", how = "left")
ocod_data["lsoa_business_counts"] = ocod_data["lsoa_business_counts"].fillna(0)

In [16]:
ocod_data[ocod_data['lsoa11cd'].isnull()].to_csv('/tf/empty_homes_data/delete_me.csv')

In [17]:
pd.crosstab(ocod_data['postcode'].notnull(), ocod_data['lsoa_building'].notnull())

lsoa_building,False,True
postcode,,
False,50152,7119
True,81217,21112


## looking at what is left
what still doesn't have lsoa and what properties do they have?

In [18]:
test = ocod_data

In [19]:
#observations localised with lsoa and/or oa
pd.crosstab(test['lsoa11cd'].notnull(),  test['oa11cd'].notnull())/ocod_data.shape[0]

oa11cd,False,True
lsoa11cd,,
False,0.092256,0.000000
True,0.213709,0.694035


In [20]:
#this is definately the problem then
pd.crosstab(test['lsoa_street'].notnull(),  test['lsoa_building'].notnull())

lsoa_building,False,True
lsoa_street,,
False,75689,16238
True,55680,11993


In [21]:
test2 = test[test['lsoa11cd'].isnull()]
pd.crosstab(test2['property_address'].str.startswith('land') , test2['lsoa_street'].notnull())

lsoa_street,False
property_address,
False,10704
True,4020


In [22]:
#this is definately the problem then
pd.crosstab(test['street_name'].notnull(),  test['lsoa11cd'].notnull())

lsoa11cd,False,True
street_name,,
False,3245,9418
True,11479,135458


In [23]:
test[test['lsoa11cd'].isnull() & test['street_name'].notnull()].to_csv('/tf/empty_homes_data/delete_me.csv')

In [24]:
#95.5% of sets have only a single lsoa, when grouped by street, town, district and locality
#when grouped by only street and district, this number is still 90%
#excluding town the number is still 0.95% but dropping locality gives a match on 91%, therefore using locality is the key
temp = price_paid_df.groupby(['street', 'district', 'lsoa11cd']).size().reset_index().groupby(['street', 'district']).size()\
.reset_index().rename(columns = {0:'counts'})

temp.groupby('counts').size()/temp.shape[0]


counts
1     0.878886
2     0.085376
3     0.019499
4     0.007090
5     0.003393
6     0.001796
7     0.001165
8     0.000746
9     0.000512
10    0.000369
11    0.000264
12    0.000200
13    0.000140
14    0.000116
15    0.000072
16    0.000069
17    0.000052
18    0.000034
19    0.000037
20    0.000026
21    0.000021
22    0.000018
23    0.000016
24    0.000021
25    0.000013
26    0.000008
27    0.000011
28    0.000005
29    0.000006
30    0.000008
31    0.000005
33    0.000003
34    0.000005
35    0.000003
37    0.000002
41    0.000002
42    0.000002
45    0.000002
54    0.000002
57    0.000002
65    0.000002
71    0.000002
74    0.000002
dtype: float64

# VOA matching businesses

The below chunk matches addresses to known businesses

## Creating a voa expander for individual LADs

In [ ]:
all_lads = ocod_data.lad11cd.unique()

matched_lads_list = []

all_lads = [x for x in all_lads if str(x) != 'nan']
#see which roads match a road in voa data set for each local authority
for target_lad in all_lads:
    print(target_lad)
    #temp['matches_business_address'] = business_address_matcher(temp['street_name'], temp['street_number'], voa_businesses, target_lad)
    matched_lads_list = matched_lads_list + [massaged_address_match(ocod_data, voa_businesses, target_lad)]

    #matched_lads_list = [massaged_address_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]


E06000031
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E07000032
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E09000001


/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types
/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001
E08000019
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


...

/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


E09000033


/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


i= 0  expand time,0.0 filter time0.001 make_dataframe_time 0.001
E09000019
i= 0  expand time,0.0 filter time0.0 make_dataframe_time 0.001


/tf/empty_homes_london/helper_functions.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_multi['number_filter'] = filter_types


In [ ]:
voa_businesses[voa_businesses['street_name2']=="elginavenue"]

In [ ]:

#each line is a matched business
ocod_data = pd.concat(matched_lads_list)
#matched_businesses.to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
pd.crosstab(ocod_data['oa_busi_building'].notnull(), ocod_data['business_address'].notnull())

# Classify property type

In [ ]:
ocod_data['class'] = np.select(
    [
        ocod_data['property_address'].str.contains(r"^(land|plot)", case = False),
        ocod_data['property_address'].str.contains(r"^([a-z\s]*)((garage)|(parking(\s)?space)|(parking space)|(car park(ing)?))", case = False),
        ocod_data['property_address'].str.contains(r"^((the airspace)|(airspace))", case = False),
        ocod_data['property_address'].str.contains(r"(penthouse|flat|apartment)", case = False),
        ocod_data['address_match']==True,
        ocod_data['property_address'].str.contains(r"(cinema)|(hotel)|(office)|centre|(\bpub)|holiday(\s)?inn|travel(\s)?lodge|(business)|(cafe)|(^shop)|( shop)|(restaurant)|(home)|(^store(s)?\b)|(^storage\b)|(company)|(ltd)|(limited)|(plc)|(retail)|(leisure)|(industrial)|(hall of)|trading|commercial|works|advertising", case = False), 
        ocod_data['property_address'].str.contains(r"^[a-z\s']+\b(land(s)?|plot(s)?)\b", case = False)==True, #land with words before it
        ocod_data['building_name'].str.contains(r'\binn$|public house|^the\s\w+\sand\s\w+|(tavern$)')==True, #pubs in various guises
        ocod_data['oa_busi_building'].notnull(),#a business building was matched
        ocod_data['business_address'].notnull(),
        ocod_data['business_counts']==0, #if there are no businesses in the oa then it is a domestic
        ocod_data['lsoa_business_counts']==0, #if there are no businesses in the lsoa then it is a domestic
        
    ], 
    [
        'land',
        'carpark',
        'airspace',
        'domestic',
        'business',
        'business',
        'land',
        'business',
        'business',
        'business',
        'domestic',
        'domestic',
    ], 
    default='unknown'
)

In [ ]:
#ocod_data.loc[ocod_data['property_address'].str.contains(r"^[a-z\s']+\b(land(s)?|plot(s)?)\b", case = False)==True,
#             'property_address'].to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
multi_class_titles = ocod_data[~ocod_data['class'].isin(['unknown', 'airspace', 'carpark']) & (ocod_data['within_larger_title']==True)].groupby(['title_number', 'class']).\
size().reset_index().groupby('title_number').size().reset_index().rename(columns={0:'counts'})

#there are no within title-ids that have more than one class. This shows that this is a very accurate way of filling in missing class data
print(multi_class_titles[multi_class_titles['counts']>1])

multi_class_titles = multi_class_titles[multi_class_titles['counts']==1]
#multi_class_titles.groupby('counts').size()

ocod_data[ocod_data['title_number'].isin(multi_class_titles['title_number'])].groupby('class').size()
#[['street_number', 'street_name','property_address', "business_address"]]

In [ ]:
#Fills out unknown class values using the known class values from the same 

temp_fill = ocod_data[~ocod_data['class'].isin(['unknown', 'airspace', 'carpark']) & (ocod_data['within_larger_title']==True)].groupby(['title_number', 'class']).\
size().reset_index()[['title_number', 'class']].drop_duplicates()

temp = ocod_data[ocod_data['title_number'].isin(temp_fill['title_number']) & (ocod_data['class']=='unknown') & (ocod_data['within_larger_title']==True)]

temp.drop('class', axis = 1, inplace = True)

temp = temp.merge(temp_fill, how = "left", on = "title_number")

ocod_data = pd.concat([temp, ocod_data[~ocod_data['unique_id'].isin(temp['unique_id'])]])


In [ ]:
#pd.crosstab(test['class'], ocod_data['oa_busi_building'].isna()==False)

In [ ]:
ocod_data.columns

In [ ]:
#if there is a street match, and the property has a street and a street number OR a building name
#Then is is a domestic property

test = ocod_data[ocod_data['class']=='unknown']
pd.crosstab((test['street_match']==True), (test['street_name'].notnull()==True) & (test['street_number'].notnull()==True) )

In [ ]:
#If street match is false but there is a street name
#The it is a domestic property
pd.crosstab((test['street_match']==True), (test['street_name'].notnull()==True))

In [ ]:
pd.crosstab((test['street_match']==True), (test['building_name'].notnull()==True))

In [ ]:
ocod_data['class2'] = np.select(
    [
        (ocod_data['class']=='unknown') & (ocod_data['street_match']==True) & (ocod_data['street_name'].notnull()==True) & (ocod_data['street_number'].notnull()==True),
        (ocod_data['class']=='unknown') & (ocod_data['street_match']==False) & (ocod_data['street_name'].notnull()==True),
        (ocod_data['class']=='unknown') & (ocod_data['building_name'].notnull()==True)
        
    ], 
    [
        'domestic',
        'domestic',
        'domestic'
    ], 
    default= ocod_data['class']
)


#fillout larger titles that are now partially tagged

temp_fill = ocod_data[~ocod_data['class2'].isin(['unknown', 'airspace', 'carpark']) & (ocod_data['within_larger_title']==True)].groupby(['title_number', 'class2']).\
size().reset_index()[['title_number', 'class2']].drop_duplicates()

temp = ocod_data[ocod_data['title_number'].isin(temp_fill['title_number']) & (ocod_data['class2']=='unknown') & (ocod_data['within_larger_title']==True)]

temp.drop('class2', axis = 1, inplace = True)

temp = temp.merge(temp_fill, how = "left", on = "title_number")

ocod_data = pd.concat([temp, ocod_data[~ocod_data['unique_id'].isin(temp['unique_id'])]])


In [ ]:
#if there is a street match, and the property has a street and a street number OR a building name
#Then is is a domestic property

test = ocod_data[ocod_data['class2']=='unknown']
print(pd.crosstab((test['street_match']==True), (test['street_name'].notnull()==True) ))

ocod_data[(ocod_data['street_name'].isnull()==True) & (ocod_data['class2']=='unknown')].to_csv('/tf/empty_homes_data/delete_me.csv')

In [ ]:
ocod_data.groupby('class').size()

In [ ]:
pd.crosstab(ocod_data['unit_type'],(ocod_data['class2']=="domestic"))

## Removing copies

Businesses, carparks and airpsace etc are classed as a single address independent of how many components they a made of.
This chunk strips down businesses that have been expanded back to a single address


In [ ]:


temp = ocod_data[~ocod_data['class'].isin(['domestic', 'unknown'])]

ocod_data = pd.concat([ocod_data[ocod_data['class'].isin(['domestic', 'unknown'])], temp.groupby('title_number').first()]).sort_values(by = "unique_id")

#when the unit type is a carparking space but the class is domestic it means that the address is a domestic property that explicitly mentions a car park
ocod_data = ocod_data[~(ocod_data['unit_type'].str.contains(r"(park|garage)") & (ocod_data['class2']=="domestic"))]


In [ ]:
ocod_data.groupby('class').size()

In [ ]:
ocod_data.groupby('class').size()/ocod_data.shape[0]

In [ ]:
#non of the unknowns have a postcode. I guess this is obvious as if there is no matching VOA postcode you are classed as domestic
#pd.crosstab(ocod_data[ocod_data['class']=="unknown"].postcode.notnull(), ocod_data[ocod_data['class']=="unknown"].street_name.notnull())

In [ ]:
pd.crosstab(ocod_data['tenure'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

In [50]:
ocod_data.loc[ (ocod_data['class2']=="unknown"),  
              ['title_number', 'building_name', 'street_number', 'street_name', 'property_address', 'price_paid', 'lsoa11cd', 'class2']].to_csv('/tf/empty_homes_data/delete_me.csv')

In [64]:
ocod_data.loc[  (ocod_data['lad11cd'].isin(['E08000011','E08000012','E08000013',  'E08000014', 'E08000015'])==True),  
              ['title_number', 'building_name', 'street_number', 'street_number2', 'street_name', 'property_address', 'price_paid', 'lsoa11cd', 'class', 'class2']].groupby('class2').size()#.to_csv('/tf/empty_homes_data/merseyside.csv')

class2
airspace     192
business     410
carpark      123
domestic    3940
land         333
unknown      100
dtype: int64

In [53]:


#ocod_data['building_name'].str.contains(r'^the\s\w+\sand\s\w+$')==True
#'\binn$'

ocod_data.loc[ ocod_data['building_name'].str.contains(r'tavern$')==True,  
              ['title_number', 'building_name', 'street_number', 'street_number2', 'street_name', 'property_address', 'price_paid', 'lsoa11cd', 'class']]#.to_csv('/tf/empty_homes_data/delete_me.csv')

,title_number,building_name,street_number,street_number2,street_name,property_address,price_paid,lsoa11cd,class
101426,NaN,the railway tavern,393,393,old ford road,"The Railway Tavern, 393 Old Ford Road, London ...",110000.0,E01004224,business
375861,NaN,the cock tavern,23,23,phoenix road,"The Cock Tavern, 23 Phoenix Road, London (NW1 ...",NaN,E01000953,business
69284,NaN,the grove tavern,83,83,hammersmith grove,"The Grove Tavern, 83 Hammersmith Grove, London...",NaN,E01001898,business
AV56717,NaN,air balloon tavern,11,11,air balloon road,"Air Balloon Tavern, 11 Air Balloon Road, St Ge...",69750.0,E01014682,business
CB271724,NaN,minster tavern,1,1,minster place,"Minster Tavern, 1 Minster Place, Ely (CB7 4EL)",NaN,E01018025,business
DY405002,NaN,railway tavern,None,nan,station road,"Railway Tavern, Station Road, Hatton, Derby (D...",NaN,E01019845,business
GM567433,NaN,town hall tavern,None,nan,wellington road south,"Town Hall Tavern ,95 Wellington Road South, St...",189290.0,E01005758,business
HS302581,NaN,saltshouse tavern,1,1,dunvegan road,"Saltshouse Tavern, 1 Dunvegan Road, Hull (HU8 ...",NaN,E01012821,business
NGL477707,NaN,the crown tavern,158,158,prince of wales road,"The Crown Tavern, 158 Prince of Wales Road, St...",200000.0,E01000905,business
NGL792560,NaN,part of the cock tavern,259,259,upper street,"Part of The Cock Tavern, 259 Upper Street, Lon...",NaN,E01002795,business


In [53]:
pd.crosstab(ocod_data['class2'], ocod_data['region'].str.lower())#.to_latex() #convert to copyable latex table

region,east anglia,east midlands,greater london,north,north west,south east,south west,wales,west midlands,yorks and humber
class2,,,,,,,,,,
airspace,50,162,87,15,215,185,266,290,94,303
business,418,914,9153,637,1717,3099,1091,469,1034,910
carpark,27,36,1454,23,518,186,88,9,49,48
domestic,2662,3749,47847,2656,16985,14501,4566,2614,4287,8386
land,639,918,2622,727,2210,4789,1251,629,1303,1311
unknown,56,110,753,128,317,458,194,77,141,295


In [ ]:
pd.crosstab(ocod_data['class2'], ocod_data['tenure'])#.to_latex() #convert to copyable latex table

In [ ]:
temp_df = ocod_data[['title_number', 'tenure', 'within_larger_title']].drop_duplicates()

#most of titles containing nested addresses are free hold by about 3/2
pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])


In [ ]:
#The analysis is based on nested addresses being domestic
temp_df = ocod_data[['title_number', 'tenure', 'property_address']][ocod_data['within_larger_title']==True]
temp_df['is_flat'] = temp_df['property_address'].str.contains(r"(flat|apartment|penthouse|unit)", case = False)

#pd.crosstab(temp_df['tenure'], temp_df['within_larger_title'])

temp_df.groupby('tenure').size()

#Of nested addresses freehold is more common by 3/2 50k to 24k
#most of theproperties are not flats however flats dominate the leasehold section
#flats are 1/3 of nested addresses but make up almost 3/4 of the leashold nested addresses
#note this does not include items marked as units
pd.crosstab(temp_df.tenure, temp_df.is_flat)

In [ ]:
#This is interesting as it shows that only about 15% of normal titles are missing a postcode whilst half of within larger title addresses are missing a postcode.
#This is probably because they cover multiple postcodes
pd.crosstab(ocod_data.postcode.notnull(), ocod_data.within_larger_title)

In [ ]:
#64% of the propertys have postcodes/lsoa.
#I need to match the remaining to get an accurate distribution of the properties

ocod_data.postcode.notnull().sum()/ocod_data.shape[0]

## Road matching

## Exploring road matching

Before matching the names I should edit the street names to remove all ' apostraphes as these are very unreliable, I should also remove spaces between 'gate' and whatever comes before it same with 'way'
It may also be an idea to remove 's' that appears at the end of a word but we can try this again later

In [ ]:
#target_lad = 'E09000019'
#test= fuzzy_street_match(ocod_data, voa_businesses, target_lad)


In [ ]:
target_lad = 'E09000019'
test= massaged_street_match(ocod_data, voa_businesses, target_lad)

test.columns#[test['street_name2']].groupby('match').size()

In [ ]:
#all_lads = ocod_data.lad11cd.unique()

#all_lads = [x for x in all_lads if str(x) != 'nan']
##see which roads match a road in voa data set for each local authority
#ocod_road_match= [fuzzy_street_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]
#ocod_road_match = pd.concat(ocod_road_match)

In [ ]:
all_lads = ocod_data.lad11cd.unique()

all_lads = [x for x in all_lads if str(x) != 'nan']
#see which roads match a road in voa data set for each local authority
ocod_road_match= [massaged_street_match(ocod_data, voa_businesses, target_lad) for target_lad in all_lads]
ocod_road_match = pd.concat(ocod_road_match)

In [ ]:
ocod_road_match.groupby('match').size()

In [ ]:
test = ocod_road_match2[ocod_road_match2['street_name'] != 'xxxstreet name missingxxx']
pd.crosstab(test['match'], test['class'])

In [2]:
498/(498+3781)

0.11638233232063566

In [3]:

ocod_road_match2[(ocod_road_match2['match']==False) & (ocod_road_match2['class'] == "business")& (ocod_road_match2['street_name'] != 'xxxstreet name missingxxx')].to_csv("/tf/empty_homes_data/fuzzy_roads.csv")

NameError: name 'ocod_road_match2' is not defined

In [49]:

ocod_road_match[(ocod_road_match['similarity']<93) & (ocod_road_match['class'] == "business") & (ocod_road_match['street_name'] != 'xxxstreet name missingxxx')].to_csv("/tf/empty_homes_data/fuzzy_roads.csv")

NameError: name 'ocod_road_match' is not defined

In [ ]:
ocod_road_match[ocod_road_match['street_name'].str.contains(r"'")][['street_name', 'street_name2', 'matched_road_name' ]]

In [ ]:
#values greater than or equal to 94 seem to be pretty good
ocod_data.columns
#ocod_road_match[ocod_road_match['similarity']==96][['property_address', 'street_name2', 'matched_road_name']]

# creating the aggregated ocod dataset for sampling

In [55]:
ocod_data_lsoa = ocod_data
#ocod_data_lsoa['postcode2'] = ocod_data['postcode'].str.lower().str.replace("\s", "")

#ocod_data_lsoa = ocod_data.merge(postcode_district_lookup, 'left', left_on = "postcode2", right_on = "postcode2")

ocod_data_lsoa.groupby(['lad11cd', 'lsoa11cd', 'msoa11cd', 'class2']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa.csv")
ocod_data_lsoa.groupby(['lad11cd', 'lsoa11cd', 'msoa11cd', 'class2', 'within_larger_title']).size().reset_index().to_csv("/tf/empty_homes_data/ocod_lsoa_by_nested_type.csv")



## Largest nested addresses

In [ ]:
#The largest nested address
ocod_data.within_title_id.max()

In [ ]:
ocod_data[ocod_data.within_title_id==ocod_data.within_title_id.max()].reset_index()['property_address'][0]



In [ ]:
ocod_data.to_csv("/tf/empty_homes_data/OCOD_classes.csv")

In [ ]:
test  = ocod_data[ocod_data['class']=="unknown" ]

pd.crosstab(test.postcode.notnull(), test.street_name.notnull())

In [ ]:
ocod_data[ ocod_data.street_name.isnull()].to_csv("/tf/empty_homes_data/OCOD_no_street.csv")

In [ ]:
1/21